In [98]:
# Render our plots inline
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('mode.chained_assignment', None)

In [4]:
log_file = pd.read_csv('log11-18.csv')

In [5]:
log_file[:3]

,Unnamed: 0,id,timestamp,sid,event,act,div_id,course_id,base_course
0,0,1760,2017-08-13 20:23:59,301304554400186198747097533764849583054@208.15...,page,view,index.html,47.0,apcsareview
1,1,1778,2017-08-13 23:14:25,del1,page,view,index.html,29.0,apcsareview
2,2,1779,2017-08-13 23:17:48,del1,page,view,VariableBasics/operators.html,29.0,apcsareview


In [27]:
pretest_logs = log_file.loc[log_file['div_id'].str.startswith('pre_', na=False)] 

In [80]:
pretest_logs['pretest_correct'] = 

In [99]:
pretest_logs['pretest_score'] = np.where(pretest_logs['act'].str.endswith('correct', na=False), 1, 0)

In [100]:
pretest_logs.head()

,Unnamed: 0,id,timestamp,sid,event,act,div_id,course_id,base_course,bidtime_hours,pretest_score,pretest_correct
305,305,10247,2017-08-18 20:42:42,271926861551350033289774502469454480334@146.16...,mChoice,answer:1:correct,pre_1,213.0,StudentCSP,271926861551350033289774502469454480334@146.16...,1,True
306,306,10248,2017-08-18 20:42:42,271926861551350033289774502469454480334@146.16...,mChoice,answer:2:correct,pre_7,213.0,StudentCSP,271926861551350033289774502469454480334@146.16...,1,True
307,307,10249,2017-08-18 20:42:43,271926861551350033289774502469454480334@146.16...,mChoice,answer:0:no,pre_5,213.0,StudentCSP,271926861551350033289774502469454480334@146.16...,0,False
308,308,10250,2017-08-18 20:42:43,271926861551350033289774502469454480334@146.16...,mChoice,answer:0:no,pre_4,213.0,StudentCSP,271926861551350033289774502469454480334@146.16...,0,False
309,309,10251,2017-08-18 20:42:43,271926861551350033289774502469454480334@146.16...,mChoice,answer:2:correct,pre_6,213.0,StudentCSP,271926861551350033289774502469454480334@146.16...,1,True


In [21]:
 1 if pretest_logs[:1]['act'].to_string().endswith('correct') else 0

TypeError: string indices must be integers